## Article Collection and Volume Reduction Pipeline 
1. Find efficient keywords with word embeddings (gensim)
2. Remove duplicitous articles with cosine similarity on TFIDF vectors (scikit-learn)
3. Remove duplicitous articles with entity extraction and jaccard similarity (spacy)
4. **Classify relevant articles** (scikit-learn)

## Summary
Using the article title and summary, we use a TFIDF matrix as features in a logistic regression to predict whether an article is about the war on terror. In practice, we fine tune a cutoff threshold to improve recall on a validation dataset.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import spacy

In [2]:
usecols = ['Article_ID', 'Title', 'Summary', 'War on Terror']
data = (pd
        .read_csv('../data/nyt_ftpg_1996_2006_no_text.csv', engine='python', usecols=usecols)
        .assign(text=lambda x: x['Title'] + ' ' + x['Summary'])
    )

In [3]:
data.head()

,Article_ID,Title,Summary,War on Terror,text
0,1,Nation's Smaller Jails Struggle To Cope With S...,Jails overwhelmed with hardened criminals,0,Nation's Smaller Jails Struggle To Cope With S...
1,2,Dancing (and Kissing) In the New Year,new years activities,0,Dancing (and Kissing) In the New Year new yea...
2,3,Forbes's Silver Bullet for the Nation's Malaise,Steve Forbes running for President,0,Forbes's Silver Bullet for the Nation's Malais...
3,4,"Up at Last, Bridge to Bosnia Is Swaying Gatewa...",U.S. military constructs bridge to help their ...,0,"Up at Last, Bridge to Bosnia Is Swaying Gatewa..."
4,5,2 SIDES IN SENATE DISAGREE ON PLAN TO END FURL...,Democrats and Republicans can't agree on plan ...,0,2 SIDES IN SENATE DISAGREE ON PLAN TO END FURL...


In [4]:
# proportion of articles with label
data['War on Terror'].value_counts(normalize=True)

0    0.863795
1    0.136205
Name: War on Terror, dtype: float64

In [5]:
nlp = spacy.load('en')
def lemmatizer(text):
    return [token.lemma_ for token in nlp(text) if not (token.is_stop or token.is_punct)]

In [6]:
tfidf = TfidfVectorizer(min_df=3, analyzer=lemmatizer)
tfidf_array = tfidf.fit_transform(data['text'])


In [7]:
# format data for input
X = tfidf_array
y = data['War on Terror'].values

# split data into training and test
tts = train_test_split(X, y, test_size=0.33, random_state=666)
X_train, X_test, y_train, y_test = tts

In [8]:
lr = LogisticRegressionCV()
lr.fit(X_train, y_train)
print("Accuracy:", lr.score(X_test, y_test))

Accuracy: 0.9620191368873267


In [9]:
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.97      0.99      0.98      8845
          1       0.90      0.81      0.85      1397

avg / total       0.96      0.96      0.96     10242



In [10]:
# adjust threshold for higher recall
threshold = 0.05
y_pred_threshold = lr.predict_proba(X_test)[:,1] > threshold
print(classification_report(y_test, y_pred_threshold))

             precision    recall  f1-score   support

          0       0.99      0.92      0.96      8845
          1       0.66      0.96      0.78      1397

avg / total       0.95      0.93      0.93     10242



In [11]:
print("confusion matrix")
pd.crosstab(pd.Series(y_test, name='actual'),
            pd.Series(y_pred_threshold, name='predicted'))

confusion matrix


predicted,False,True
actual,,
0,8162,683
1,55,1342


In [12]:
print("Proportion of Articles above threshold:", sum(y_pred_threshold) / len(y_test))

Proportion of Articles above threshold: 0.1977152899824253
